In [1]:
#!pip install requests-html

In [32]:
from bs4 import BeautifulSoup
import requests
import time
import datetime

import smtplib


In [33]:
# Connect to Website and pull in data

URL = 'https://www.amazon.com/s?k=science+books&crid=1KNJQBAGX4FOF&sprefix=science+boo%2Caps%2C228&ref=nb_sb_ss_deep-retrain-ln-ops-acceptance_2_11'

#headers info below (User-Agent for this computer) is taken from site -> https://httpbin.org/get
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate, br", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

s = BeautifulSoup(page.content, "html.parser")

soup = BeautifulSoup(s.prettify(), "html.parser")

allsection = soup.find('div', class_="a-section a-spacing-base")

In [34]:
# #alternative way to connect to website and pull data - does not require headers
# #Does not work - unable to pass amazon authomatic detection
# ##To discuss automated access to Amazon data please contact api-services-support@amazon.com.

# from bs4 import BeautifulSoup
# from requests_html import HTMLSession

# session = HTMLSession()
# URL = 'https://www.amazon.com/s?k=science+books&crid=1KNJQBAGX4FOF&sprefix=science+boo%2Caps%2C228&ref=nb_sb_ss_deep-retrain-ln-ops-acceptance_2_11'
# s = session.get(URL)
# Soup = BeautifulSoup(s.text, 'html.parser')

# print(Soup)

In [35]:
#fina alla books on a page, pull info on those books -> title, price and author, and write row per each book into csv file

import csv
from csv import writer

#creating a csv file, with column's headers
csv_file=open('amazom_book_search.csv','w')
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['title', 'price', 'author', 'date'])

def scrape_from_page (URL):
    #headers info below (User-Agent for this computer) is taken from site -> https://httpbin.org/get
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate, br", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    s = BeautifulSoup(page.content, "html.parser")

    soup = BeautifulSoup(s.prettify(), "html.parser")
    allsection = soup.find_all('div', class_="a-section a-spacing-base")
    date_of_search=datetime.date.today()
    
    for section in allsection:
        #add to next lines check if None, and also check why there is frequently no title and author
        try:
            title= section.find('span', class_="a-size-base a-color-base a-text-normal").get_text().strip()
        except AttributeError:
            continue
        try:
            price = section.find('span', class_="a-offscreen").get_text().strip().replace(" ","").replace("\n","")
        except AttributeError:
            price='none'

        try:
            author = section.find('a', class_= "a-size-base a-link-normal puis-light-weight-text s-underline-text s-underline-link-text s-link-style s-link-centralized-style").contents[0].strip()
        except AttributeError:
            author='none'

        #adding and saving to csv file
        csv_file=open('amazom_book_search.csv','a', encoding="utf-8")
        csv_writer=csv.writer(csv_file)
        csv_writer.writerow([title, price, author, date_of_search])
        csv_file.close()

In [24]:
#calling he function scrape_from_page
#URL='https://www.amazon.com/s?k=science+books&page=5&crid=1KNJQBAGX4FOF&qid=1665252034&sprefix=science+boo%2Caps%2C228&ref=sr_pg_4'
scrape_from_page(URL)

In [37]:
#mooving through pages

URL='https://www.amazon.com/s?k=science+books&crid=1KNJQBAGX4FOF&sprefix=science+boo%2Caps%2C228&ref=nb_sb_ss_deep-retrain-ln-ops-acceptance_2_11'
print(URL)
page = requests.get(URL, headers=headers)
s = BeautifulSoup(page.content, "html.parser")
page_=s.find('div', class_="a-section a-text-center s-pagination-container")

while URL != 'last page':
    if not page_.find('span', class_="s-pagination-item s-pagination-next s-pagination-disabled"):
        page = requests.get(URL, headers=headers)
        s = BeautifulSoup(page.content, "html.parser")
        page_=s.find('div', class_="a-section a-text-center s-pagination-container")
        pageend=page_.find('span', class_='s-pagination-strip').find_all('a')[-1]['href']
        
        scrape_from_page(URL)
        
        URL='https://www.amazon.com'+str(pageend)
        print('one page finnished')
        print(pageend)
        
    else:
        print('last page')
        URL='last page'



https://www.amazon.com/s?k=science+books&crid=1KNJQBAGX4FOF&sprefix=science+boo%2Caps%2C228&ref=nb_sb_ss_deep-retrain-ln-ops-acceptance_2_11
one page finnished
/s?k=science+books&page=2&crid=1KNJQBAGX4FOF&qid=1665252625&sprefix=science+boo%2Caps%2C228&ref=sr_pg_1
one page finnished
/s?k=science+books&page=3&crid=1KNJQBAGX4FOF&qid=1665252629&sprefix=science+boo%2Caps%2C228&ref=sr_pg_2
one page finnished
/s?k=science+books&page=4&crid=1KNJQBAGX4FOF&qid=1665252633&sprefix=science+boo%2Caps%2C228&ref=sr_pg_3
one page finnished
/s?k=science+books&page=5&crid=1KNJQBAGX4FOF&qid=1665252637&sprefix=science+boo%2Caps%2C228&ref=sr_pg_4
one page finnished
/s?k=science+books&page=6&crid=1KNJQBAGX4FOF&qid=1665252641&sprefix=science+boo%2Caps%2C228&ref=sr_pg_5
one page finnished
/s?k=science+books&page=7&crid=1KNJQBAGX4FOF&qid=1665252645&sprefix=science+boo%2Caps%2C228&ref=sr_pg_6
one page finnished
/s?k=science+books&page=6&crid=1KNJQBAGX4FOF&qid=1665252649&sprefix=science+boo%2Caps%2C228&ref=sr_pg_

In [38]:
#displaying out csv data in dataframe for possible future analysis
import pandas as pd
booksdata_amazon=pd.read_csv(r'C:\Users\yelsk\amazom_book_search.csv')

#len(booksdata_amazon['title'].unique())
booksdata_amazon

,title,price,author,date
0,The Origin of Species,$13.99,Charles Darwin,2022-10-08
1,Quantum Physics For Beginners: The Simple Guid...,$13.95,Jeremy Wooster,2022-10-08
2,5000 Amazing Facts For Kids: Learning Facts on...,$15.12,none,2022-10-08
3,Fundamentals of Drug Development,$105.95,none,2022-10-08
4,The Fascinating Science Book for Kids: 500 Ama...,$10.99,Kindle,2022-10-08
...,...,...,...,...
265,Science Comics: Plagues: The Microscopic Battl...,$7.98,none,2022-10-08
266,Galaxies: Birth and Destiny of Our Universe,$44.99,none,2022-10-08
267,Saving Nature (Adventures with Buku),$12.99,none,2022-10-08
268,How To Build A Dragon Or Die Trying: A Satiric...,$21.99,none,2022-10-08


In [74]:
#cleaning and changing data types

#booksdata_amazon['price'] = booksdata_amazon['price'].str.strip('$')

booksdata_amazon['price'] = pd.to_numeric(booksdata_amazon['price'],errors='coerce')
booksdata_amazon['date']= pd.to_datetime(booksdata_amazon['date'], format='%Y-%m-%d')
#print (booksdata_amazon.dtypes)
booksdata_amazon.head()

title             object
price            float64
author            object
date      datetime64[ns]
dtype: object


,title,price,author,date
0,The Origin of Species,13.99,Charles Darwin,2022-10-08
1,Quantum Physics For Beginners: The Simple Guid...,13.95,Jeremy Wooster,2022-10-08
2,5000 Amazing Facts For Kids: Learning Facts on...,15.12,none,2022-10-08
3,Fundamentals of Drug Development,105.95,none,2022-10-08
4,The Fascinating Science Book for Kids: 500 Ama...,10.99,Kindle,2022-10-08


In [98]:
#data analysis, manipulating the data
print(f"min book price is: {booksdata_amazon['price'].min()}, max price is : {booksdata_amazon['price'].max()}")
print(f"average book price is: {booksdata_amazon['price'].mean()}")

print(f"number of books found in a search (in dataframe) is: {len(booksdata_amazon)}")

if booksdata_amazon['title'].str.contains("Physics").any()==True:
    print("There were Physics books found")
else:
    print('No Physic books in a list')

phys_count=0
for title in booksdata_amazon['title']: 
    if "Physics" in title:
        phys_count+=1
    
print(f"There were {phys_count} Physic books found")

print(f"number of books on phisics found is: {booksdata_amazon['price'].mean()}")

min book price is: 0.0, max price is : 195.68
average book price is: 21.49387596899219
number of books found in a search (in dataframe) is: 270
There were Physics books found
There were 9 Physic books found
number of books on phisics found is: 21.49387596899219


In [100]:
#!pip install openpyxl
#save dataframe booksdata_amazon as excel
# create excel writer object
writer = pd.ExcelWriter('booksdata_amaz.xlsx')
# write dataframe to excel
booksdata_amazon.to_excel(writer)
# save the excel
writer.save()
